In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from autogluon import TabularPrediction as task

In [3]:
DATA_DIR = os.path.abspath('../../Data/home_credit_default_risk')
MODEL_DIR = os.path.abspath('models')

# Automated Modeling with *autogluon*

In [4]:
feature_matrix = pd.read_csv(os.path.join(DATA_DIR, 'feature_matrix.csv'), low_memory=False) 

In [5]:
df_train = feature_matrix[feature_matrix['dataset'] == 'train'].drop(['SK_ID_CURR', 'dataset'], axis=1)
df_test = feature_matrix[feature_matrix['dataset'] == 'test'].drop(['SK_ID_CURR', 'dataset', 'TARGET'], axis=1)
train_data = task.Dataset(df_train)

In [6]:
train_data.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,SUM(credit_card_balance.previous_application.DAYS_LAST_DUE),SUM(credit_card_balance.previous_application.DAYS_LAST_DUE_1ST_VERSION),SUM(credit_card_balance.previous_application.DAYS_TERMINATION),SUM(credit_card_balance.previous_application.HOUR_APPR_PROCESS_START),SUM(credit_card_balance.previous_application.NFLAG_INSURED_ON_APPROVAL),SUM(credit_card_balance.previous_application.NFLAG_LAST_APPL_IN_DAY),SUM(credit_card_balance.previous_application.RATE_DOWN_PAYMENT),SUM(credit_card_balance.previous_application.RATE_INTEREST_PRIMARY),SUM(credit_card_balance.previous_application.RATE_INTEREST_PRIVILEGED),SUM(credit_card_balance.previous_application.SELLERPLACE_AREA)
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,2191458.0,2191458.0,2191458.0,90.0,0.0,6.0,0.0,0.0,0.0,-6.0
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
excluded_model_types=['RF', 'XT']eval_metric='roc_auc', visualizer='tensorboard', 

In [26]:
predictor = task.fit(train_data=train_data.iloc[:10000, :200], label='TARGET', output_directory=MODEL_DIR, 
                     auto_stack=True, verbosity=4, 
                     hyperparameters = {'RF': [{'n_jobs': 10}]})

Beginning AutoGluon training ...
AutoGluon will save models to /home/ubuntu/Projects/home_credit_default_risk/models/
AutoGluon Version:  0.0.15
Train Data Rows:    10000
Train Data Columns: 199
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    113300.36 MB
	Train Data (Original)  Memory Usage: 31.59 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeature

			Types of features in processed data (raw dtype, special dtypes):
				('category', []) :  19 | ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', ...]
				('float', [])    : 141 | ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', ...]
				('int', [])      :  34 | ['CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'FLAG_EMP_PHONE', ...]
			0.1s = Fit runtime
			194 features in original data used to generate 194 features in processed data.
	Useless Original Features (Count: 5): ['FLAG_MOBIL', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (exact raw dtype, raw dtype):
		('float64', 'float') : 141 | ['AMT_INC

In [ ]:
lboard = predictor.leaderboard()
lboard.sort_values(by='score_val', ascending=False)